# NeRF Evaluation

In [12]:
import pathlib

import torch
import numpy as np
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image

from neural_orientation_field.nerf.dataset import NeRFPriorImageDataset
from neural_orientation_field.nerf.model import NeRfCoarseModel, NeRfFineModel
from neural_orientation_field.nerf.utils import cam_ray_from_pose, nerf_image_render

In [2]:
# Use MPS device.
USE_DEVICE = "mps"

if USE_DEVICE == "mps" and torch.mps.is_available():
    device = torch.device("mps")
elif USE_DEVICE == "cuda" and torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

device

device(type='mps')

## Load Evaluation Dataset

In [11]:
# Input
IMAGE_PATH = "../../data/images/blender-hair-long-test/rendered/"
CAMERA_PATH = "../../data/camera/blender-hair-long-test/"
CHECKPOINT_PATH = "../../data/output/nerf/blender-hair-long/"
OUTPUT_PATH = "../../data/eval/nerf/blender-hair-long"

image_path = pathlib.Path(IMAGE_PATH).resolve()
if not image_path.exists():
    raise FileNotFoundError("Image path doesn't exist.")
camera_path = pathlib.Path(CAMERA_PATH).resolve()
if not camera_path.exists():
    raise FileNotFoundError("Camera path doesn't exist.")
checkpoint_path = pathlib.Path(CHECKPOINT_PATH).resolve()
if not checkpoint_path.exists():
    raise FileNotFoundError("Checkpoint path doesn't exist.")
output_path = pathlib.Path(OUTPUT_PATH).resolve()
if not output_path.exists():
    output_path.mkdir(parents=True, exist_ok=True)

frame_name_path = camera_path / "frame-names.txt"
cam_transform_path = camera_path / "camera-transforms.npy"
cam_param_path = camera_path / "camera-params.npy"

image_path, frame_name_path, cam_transform_path, cam_param_path, checkpoint_path, output_path

(PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/images/blender-hair-long-test/rendered'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/camera/blender-hair-long-test/frame-names.txt'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/camera/blender-hair-long-test/camera-transforms.npy'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/camera/blender-hair-long-test/camera-params.npy'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/output/nerf/blender-hair-long'),
 PosixPath('/Users/fangjun/Documents/stanford/cs229/final-project/data/eval/nerf/blender-hair-long'))

In [4]:
# Load dataset.
with open(frame_name_path, "r") as frame_path_file:
    frame_names = frame_path_file.read().split("\n")
    frame_paths = [image_path / frame_name for frame_name in frame_names]
with open(cam_transform_path, "rb") as cam_transform_file:
    cam_transforms = np.load(cam_transform_file)
with open(cam_param_path, "rb") as cam_param_file:
    cam_params = np.load(cam_param_file)
image_dataset = NeRFPriorImageDataset(frame_paths, cam_params, cam_transforms)

## Load Model

In [6]:
model_params_file_name = "model_params.pth"
coarse_model_file_name = "coarse_final.pth"
fine_model_file_name = "fine_final.pth"

# Load model params.
model_params = torch.load(checkpoint_path / model_params_file_name, weights_only=True)

coarse_pos_encode = model_params["coarse_pos_encode"]
fine_pos_encode = model_params["fine_pos_encode"]
nc = model_params["nc"]
fc = model_params["fc"]
samples_per_ray = model_params["samples_per_ray"]
max_subd_samples = model_params["max_subd_samples"]

# Load model.
coarse_model = NeRfCoarseModel(num_encoding_functions=coarse_pos_encode)
coarse_model.to(device)
fine_model = NeRfFineModel(num_encoding_functions=fine_pos_encode)
fine_model.to(device)

coarse_model.load_state_dict(torch.load(checkpoint_path / coarse_model_file_name, weights_only=True))
fine_model.load_state_dict(torch.load(checkpoint_path / fine_model_file_name, weights_only=True))

<All keys matched successfully>

## Evaluate the Model

In [7]:
# Runtime parameters.
ray_batch_size = 16384

In [19]:
test_images = []
pred_images = []
for test_image, cam_transform, (h, w), (f, cx, cy) in tqdm(image_dataset):
    cam_orig, cam_ray_world = cam_ray_from_pose(cam_transform, h, w, f, cx, cy)
    _, fine_pred = nerf_image_render(
        coarse_model,
        fine_model,
        cam_orig,
        cam_ray_world,
        ray_batch_size,
        nc,
        fc,
        samples_per_ray,
        max_subd_samples,
        coarse_pos_encode,
        fine_pos_encode,
        device
    )
    test_images.append(test_image)
    pred_images.append(fine_pred)

  0%|          | 0/16 [00:00<?, ?it/s]

In [21]:
for i in tqdm(range(len(test_images))):
    test_image = Image.fromarray((test_images[i] * 255).astype(np.uint8))
    test_image.save(output_path / f"test_{i}.png")
    pred_image = Image.fromarray((pred_images[i].detach().numpy() * 255).astype(np.uint8))
    pred_image.save(output_path / f"pred_{i}.png")

  0%|          | 0/16 [00:00<?, ?it/s]